In [1]:
#IMPORTS
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.spatial.transform import Rotation as R

import dataUtils as du

In [4]:
#Create np array sampling some range with some stepsize for each angle and translation

# def get_input(max_move,step_size):
#  arr = np.arange(-max_move,max_move+step_size,step_size)
#  return arr
#   # cur = 0
#   # moves = []
#   # for i in range(steps):
#   #   temp = [0,0,0,0,0,0]
#   #   temp[move_type] = cur

#   #   moves.append()
# a = get_input(7,1)
# print(len(a))
# print(a)
def getMoveRange(inc,max_val,move_type):
  cur = -max_val
  moves = []
  while cur <= max_val:
    tmp = [0,0,0,0,0,0]
    tmp[move_type] = cur
    moves.append(tmp)
    cur += inc
  moves.append([0,0,0,0,0,0])
  return moves

x_trans = getMoveRange(0.5,10,0)
y_trans = getMoveRange(0.5,10,1)
z_trans= getMoveRange(0.5,10,2)
x_rot = getMoveRange(0.5,10,3)
y_rot = getMoveRange(0.5,10,4)
z_rot = getMoveRange(0.5,10,5)

data = x_trans + y_trans + z_trans + x_rot + y_rot + z_rot

252

In [124]:
# Create the model
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        #self.bn1 = nn.BatchNorm1d(6)
        self.fc1 = nn.Linear(input_size, hidden_size)
        #self.bn2 = nn.BatchNorm1d(30)
        self.dropout1 = nn.Dropout(p=0.1)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        #self.bn3 = nn.BatchNorm1d(30)
        self.dropout2 = nn.Dropout(p=0.1)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        #self.bn4 = nn.BatchNorm1d(30)
        self.dropout3 = nn.Dropout(p=0.1)
        self.fc4 = nn.Linear(hidden_size, output_size)

        self.dropout2 = nn.Dropout(p=0.1)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        #self.fc5 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        #x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        #x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        #x = self.dropout3(x)
        x = self.relu(self.fc5(x))
        x = self.fc4(x)
        return x

model = torch.load("models/4hid_32_neu_100epoch_p001lr_norm-11_20rev.pth")#_random20
# Define loss function and optimizer
criterion = nn.MSELoss()
L1 = nn.L1Loss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

batch_size = 1

In [125]:
# # OUR MODEL TAKES ROT THEN TRANS, ABOVE DATA IS TRANS THEN ROT. WE NEED TO ACCOUNT FOR THIS

# #Join
# data = np.array(data)
# print(data.shape)


In [126]:
path_base = "../Data/"
a = 100 #acceleration in steps
hz = 30
samples = 3 #steps in gridsearch
stepsize = 20
full_path = path_base + "phantom_data_20step_markerAndS1Correct.csv"
num_meta_rows = 6

#load all data
df = du.load_dataset(a,samples,stepsize,path_base,full_path)
#print(df.head)

# Change rotations to accurate coordinate system and make translations relative to frame 0
df = du.fixCoordinates(df)

#get valid frames (frames without wobeling/ocscilation)
start_offset = 80 #90 #(frame before action) #ORIGINAL = 65, 10step = 90, random20 = 83, 5step = 90
wanted_frames = samples ** 6 #amount of frames to extract
frame_offset = 2*hz #time between moves
spacing = 10 #how many previous frames that have to be still
t_eps = 0.5 #translational error
r_eps =0.5 #rotational error

df_valid, valid_idx = du.getValidFrames(df,wanted_frames,frame_offset,start_offset,spacing,t_eps,r_eps)
X_norm = df_valid

TOTAL FRAMES: 729 (729)
INVALID FRAMES: 4, VALID FRAMES: 725


In [127]:
# max = df_valid.max().values[2:]
# min = df_valid.min().values[2:]
# print(max)
# print(min)

In [128]:
#Normalize data
col = ["X_trans","Y_trans","Z_trans","X_rot","Y_rot","Z_rot"]

#data = np.array([[0,0,0,0,0,0],[0,0,0,5,0,0]])
data = np.array([[0,0,0,-10,0,0],[0,0,0,-5,0,0],[0,0,0,-3,0,0],[0,0,0,0,0,0],[0,0,0,3,0,0],[0,0,0,5,0,0],[0,0,0,10,0,0]])
#0,data = np.array([[0,0,0,0,-10,0],[0,0,0,0,-5,0],[0,0,0,0,-3,0],[0,0,0,0,0,0],[0,0,0,0,3,0],[0,0,0,0,5,0],[0,0,0,0,10,0]])
data_norm = np.zeros(data.shape)

for i in range(6):
  data_norm[:,i] = 2 * (data[:,i] - X_norm[col[i]].min()) / (X_norm[col[i]].max() - X_norm[col[i]].min()) - 1


print(np.min(data_norm),np.max(data_norm))


-1.586918114438728 1.4132700723652887


In [129]:
data_tensor = torch.tensor(data_norm, dtype=torch.float32) #data_norm

# Combine the input data and target values into a TensorDataset
data_set = TensorDataset(data_tensor)

# Create a DataLoader for batch processing
data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=False)

In [130]:
def toNumpy(tensors):
  arr = [tensor.numpy() for tensor in tensors]

  # Create a NumPy array containing these NumPy arrays
  arr = np.array(arr)
  return arr

In [131]:
# Make setup that takes 1 person and gives cable lengths corresponding to that person

preds = []
# Validation loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for inputs in data_loader:
        outputs = model(inputs[0])
        preds.append(outputs)

#print(preds[0])
preds = toNumpy(preds)
print(preds)
#preds = preds.reshape(10,380,6)

[[[-5.5623318e+01  9.3809700e+00 -7.7673879e+00 -1.2137777e+01
    8.8490047e+00 -8.5902176e+01]]

 [[-3.5869339e+01  7.7797608e+00 -3.1689639e+00 -7.8364973e+00
    8.2932577e+00 -5.8460491e+01]]

 [[-2.0573063e+01  7.1100774e+00 -1.5568756e+00 -5.1178102e+00
    8.4943056e+00 -4.0112709e+01]]

 [[ 2.0918477e+00 -3.5554200e-02  3.1788478e+00  2.3144820e-01
    1.7708586e-01  1.2558415e+00]]

 [[ 8.2266655e+00 -1.0648525e+01  4.8272605e+00  4.2475939e+00
   -1.0924109e+01  8.0278873e+00]]

 [[ 1.1983497e+01 -1.6989828e+01  5.3511472e+00  6.9090261e+00
   -1.8326746e+01  1.2347897e+01]]

 [[ 1.8983227e+01 -2.9210548e+01  2.9990230e+00  1.3210029e+01
   -3.2825665e+01  2.2647940e+01]]]


In [132]:
#round each step and cast them to ints, also include a reset step at last
data_steps = np.round(preds).astype(int)

print(data_steps.shape)
#10,380,6
#tps = []

# for i in range(data_steps.shape[0]):
#   l = list(map(list,data_steps[i]))
#   l.append([0,0,0,0,0,0])
#   tps.append(l)

#print(len(tps))
data_steps = data_steps.reshape((2,6))
def inspect_tp_vals(tp,min_val,max_val):
  print(f"MAX {np.max(np.array(tp),axis=0)} MIN {np.min(np.array(tp),axis=0)}")
  for s,i in enumerate(tp):
    for j in i:
      if j > max_val or j < min_val:
        print(i,s)
        break

#for i in range(len(tps)):
#print(f"######## tp {i} ########")
#tps = tps.r
tps = list(map(list,data_steps))
inspect_tp_vals(tps,-20,20)

#np.save("data/person_data/full_3hid_32_neu_500epoch_p001lr_norm-11_nod100.npy",np.array(tps))
#arr = np.array([1,2,3,4])
#np.save("data/person_data/test.npy",arr)

(7, 1, 6)


ValueError: cannot reshape array of size 42 into shape (2,6)

In [ ]:
print(tps)

[[2, 1, -2, 1, 0, -2], [10, -14, 7, 11, -16, 11]]


In [ ]:
# test = np.load("data/person_data/full_3hid_32_neu_500epoch_p001lr_norm-11.npy")
# print(test.shape)
# print(test)